In [1]:
ride_data = "2025-04-20_10_22_12_my_iOS_device_SAMPLE.csv"
ride_data = "2025-04-20_10_22_12_my_iOS_device.csv"

outfile = "map.html"

In [2]:
import pandas as pd
import folium
from folium.plugins import AntPath
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, filtfilt

In [3]:
df = pd.read_csv(ride_data)

In [7]:


# === High-pass filter ===
def highpass_filter(data, cutoff=1.0, fs=50, order=3):
    nyq = 0.5 * fs
    norm_cutoff = cutoff / nyq
    b, a = butter(order, norm_cutoff, btype='high', analog=False)
    return filtfilt(b, a, data)

# === Prepare sensor data ===
sensor_data = df[[
    'accelerometerAccelerationX(G)',
    'accelerometerAccelerationY(G)',
    'accelerometerAccelerationZ(G)',
    'gyroRotationX(rad/s)',
    'gyroRotationY(rad/s)',
    'gyroRotationZ(rad/s)',
    'locationSpeed(m/s)'
]].copy()

sensor_data.fillna(method='ffill', inplace=True)

# High-pass filter accelerometer axes
for axis in ['X', 'Y', 'Z']:
    col = f'accelerometerAcceleration{axis}(G)'
    sensor_data[f'hpf_{axis}'] = highpass_filter(sensor_data[col].fillna(0))

# Compute vibration energy
sensor_data['vibration_energy'] = (
    sensor_data['hpf_X']**2 +
    sensor_data['hpf_Y']**2 +
    sensor_data['hpf_Z']**2
)

# === Feature engineering ===
window_size = 20
features = pd.DataFrame()
features['vibration_energy_avg'] = sensor_data['vibration_energy'].rolling(window=window_size).mean()
features['speed_avg'] = sensor_data['locationSpeed(m/s)'].rolling(window=window_size).mean()
features['gyro_variability'] = sensor_data[[
    'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)'
]].rolling(window=window_size).std().sum(axis=1)

# NEW: Normalize vibration by speed
features['vibration_per_speed'] = features['vibration_energy_avg'] / (features['speed_avg'] + 0.1)

features = features.dropna()

# === Clustering ===
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=3, random_state=42)
features['surface_type'] = kmeans.fit_predict(features_scaled)

# === Assign labels based on speed-adjusted vibration ===
results = df.iloc[window_size - 1:].copy()
results['surface_type'] = features['surface_type'].values

cluster_vibration = features.groupby('surface_type')['vibration_per_speed'].mean().sort_values()

label_mapping = {
    cluster_vibration.index[0]: 'Road',
    cluster_vibration.index[1]: 'Gravel',
    cluster_vibration.index[2]: 'Rock'
}

results['terrain_type'] = results['surface_type'].map(label_mapping)

# === Create animated map with compressed segments ===
center_lat = results['locationLatitude(WGS84)'].mean()
center_lon = results['locationLongitude(WGS84)'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=15)
color_map = {'Road': 'green', 'Gravel': 'orange', 'Rock': 'red'}

# Downsample for speed
plot_results = results.iloc[::10].reset_index(drop=True)

segment = []
last_terrain = None

for row in plot_results.itertuples():
    terrain = row.terrain_type
    latlon = [row._4, row._5]  # lat/lon

    if terrain != last_terrain and segment:
        AntPath(
            locations=segment,
            color=color_map.get(last_terrain, 'gray'),
            weight=6,
            opacity=0.8,
            dash_array=[10, 20],
            delay=500
        ).add_to(m)
        segment = []

    segment.append(latlon)
    last_terrain = terrain

# Final segment
if segment:
    AntPath(
        locations=segment,
        color=color_map.get(last_terrain, 'gray'),
        weight=6,
        opacity=0.8,
        dash_array=[10, 20],
        delay=200
    ).add_to(m)

m.save(outfile)
print(f"Map saved to {outfile}")


/var/folders/t0/4yhm4qgs2yz4b137fjy0bk_00000gn/T/ipykernel_18092/4268305063.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sensor_data.fillna(method='ffill', inplace=True)


Map saved to map.html
